In [ ]:
from datetime import datetime
import time 
import requests 
import json
import pandas as pd
#
import matplotlib.pyplot as plt  
import warnings 

In [ ]:
#Creds
my_key = 'c0c686f666abd37a38a025f1902a0a176439795eb563712822570beffacaada3'

In [ ]:
def epoch_to_strftime(epoch):
    return str(datetime.fromtimestamp(epoch).date())

def DATE(date_str):
    temp = date_str.split('-')
    return datetime(int(temp[0]),int(temp[1]),int(temp[2]))

def DATE_epoch(date_str):
    return DATE(date_str).timestamp()

def Get_OHLCV(my_key,toTs = None,
                limit = 1000,
                TSYM = 'USD',
                FSYM = 'BTC'):
    if type(toTs) != int and toTs != None:
        toTs = DATE_epoch(toTs)
    assert limit <= 2000,'Search Limit MUST be less than 2000'
    daily_uri = f'https://min-api.cryptocompare.com/data/v2/histoday?fsym={FSYM}&tsym={TSYM}&limit={limit}&api_key={my_key}'
    if toTs is not None:
        daily_uri += f'&toTs={toTs}'
    res = requests.get(daily_uri).json()
    with open('_Bit_Coin_Hist_Price.json','w') as out_file:  
        json.dump(res['Data']['Data'],out_file)
    BTC_df = pd.read_json('_Bit_Coin_Hist_Price.json')
    BTC_df['date'] = BTC_df.time.map(epoch_to_strftime)
    return BTC_df 
    
def Get_fromTs(respnose):
    Res_dct = respnose.json()
    return epoch_to_strftime(Res_dct['Data']['TimeFrom']) 

def Get_date_till(my_key,target_date:str):
    df_frac = Get_OHLCV(my_key)
    df_Final = df_frac.copy()
    target = DATE(target_date)
    toTs = df_frac.date[0]
    while DATE(toTs) > target:
        df_frac = Get_OHLCV(my_key,toTs)
        toTs = df_frac.date[0]
        print(f"Current From {toTs}")
        df_Final = df_Final.append(df_frac)
    return df_Final.drop(['conversionSymbol'],axis=1).sort_values('date').reset_index(drop=True)

In [ ]:
df_Final = Get_date_till(my_key,'2014-04-09')

Current From 2016-09-03
Current From 2013-12-07


In [ ]:
df_Final.sort_values('date').to_csv('BitCoin_hist_data.csv')